In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = "https://fr.wikipedia.org/wiki/Liste_des_pays_par_PIB_nominal"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Accept-Language": "en-US,en;q=0.9",
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except Exception as e:
    print("Failed to retrieve the webpage", response.status_code)
    exit()


In [3]:
response =requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
table = soup.find_all('table', class_="sticky-header-multi")
data = []

In [ ]:
pib_1980_1989 = table[0].text
pib_1990_1999 = table[1].text
pib_2000_2009 = table[2].text
pib_2010_2019 = table[3].text
pib_2020_2023 = table[4].text

In [43]:
def convert_to_dataframe(data_string):
    """
    Converts a string to a pandas DataFrame
    """
    # Split lines and clean
    lines = [line.strip() for line in data_string.split('\n') if line.strip()]
    
    # The first 11 lines are the columns (Country/Zone + 10 years)
    columns = lines[:11]
    
    # The rest is data
    data = []
    i = 11
    while i < len(lines):
        country = lines[i]
        # Take the next 10 values (for the 10 years)
        values = lines[i+1:i+11]
        data.append([country] + values)
        i += 11
    
    # Create pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
df_1980_1989 = convert_to_dataframe(pib_1980_1989)
df_1990_1999 = convert_to_dataframe(pib_1990_1999)
df_2000_2009 = convert_to_dataframe(pib_2000_2009)
df_2010_2019 = convert_to_dataframe(pib_2010_2019)

In [46]:
df_2000_2009.head()

,Pays/Zone,PIB 2000,PIB 2001,PIB 2002,PIB 2003,PIB 2004,PIB 2005,PIB 2006,PIB 2007,PIB 2008,PIB 2009
0,Afghanistan,-,-,"4,367","4,553","5,146","6,167","6,925","8,556","10,297","12,066"
1,Albanie,"3,483","3,928","4,348","5,611","7,185","8,052","8,896","10,677","12,881","12,044"
2,Algérie,"54,749","54,745","56,761","67,864","85,332","103,198","117,027","134,977","171,001","137,211"
3,Andorre,"1,429","1,547","1,758","2,362","2,896","3,158","3,456","3,952","4,082","3,675"
4,Angola,"11,166","10,93","15,286","17,813","23,552","36,971","52,381","65,266","88,539","70,307"


In [53]:
# Séparer les lignes et nettoyer
lignes = [ligne.strip() for ligne in pib_2020_2023.split('\n') if ligne.strip()]

# Les 11 premières lignes sont les colonnes (Pays/Zone + 10 années)
colonnes = lignes[:5]

# Le reste sont les données
donnees = []
i = 5
while i < len(lignes):
    pays = lignes[i]
    # Prendre les 10 valeurs suivantes (pour les 10 années)
    valeurs = lignes[i+1:i+5]
    donnees.append([pays] + valeurs)
    i += 5  # On saute de 12 (1 pays + 10 valeurs + 1 ligne vide)

# Créer le DataFrame pandas
df_2020_2023 = pd.DataFrame(donnees, columns=colonnes)
df_2020_2023.head()

,Pays/Zone,PIB 2020,PIB 2021,PIB 2022,PIB 2023
0,Afghanistan,"20,136","14,941",-,-
1,Albanie,"15,192","17,984","19,083","23,032"
2,Algérie,"145,656","163,138","195,06","224,107"
3,Andorre,"2,885","3,325","3,352","3,692"
4,Angola,"57,139","74,861","122,781","93,796"


In [59]:
full_pib_df = pd.merge(df_1980_1989, df_1990_1999, on="Pays/Zone", how="outer")
full_pib_df = pd.merge(full_pib_df, df_2000_2009, on="Pays/Zone", how="outer")
full_pib_df = pd.merge(full_pib_df, df_2010_2019, on="Pays/Zone", how="outer")
full_pib_df = pd.merge(full_pib_df, df_2020_2023, on="Pays/Zone", how="outer")
full_pib_df.head()

,Pays/Zone,PIB 1980,PIB 1981,PIB 1982,PIB 1983,PIB 1984,PIB 1985,PIB 1986,PIB 1987,PIB 1988,...,PIB 2014,PIB 2015,PIB 2016,PIB 2017,PIB 2018,PIB 2019,PIB 2020,PIB 2021,PIB 2022,PIB 2023
0,Afghanistan,-,-,-,-,-,-,-,-,-,...,"20,616","20,057","18,02","18,883","18,401","18,876","20,136","14,941",-,-
1,Afrique,"465,984","475,726","481,617","462,898","404,801","397,355","436,64","489,89","520,383",...,"2 650,979","2 422,14","2 303,976","2 317,348","2 482,1","2 542,636","2 457,79","2 745,183","2 945,709","2 858,342"
2,Afrique du Sud,"89,412","93,15","85,897","96,205","84,838","64,481","73,399","96,533","104,01",...,"381,195","346,663","323,493","381,317","405,093","389,245","338,193","420,009","405,106","380,906"
3,Albanie,"1,946","2,229","2,296","2,319","2,29","2,339","2,587","2,566","2,53",...,"13,246","11,389","11,862","13,053","15,157","15,399","15,192","17,984","19,083","23,032"
4,Algérie,"42,346","44,372","44,78","47,529","51,513","61,132","61,535","63,3","51,664",...,"213,81","165,979","160,034","167,498","174,868","171,673","145,656","163,138","195,06","224,107"


In [60]:
full_pib_df.to_csv("pib_data.csv", index='False', encoding="utf-8")